In [1]:
#import statements, hyper params, models
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.pyplot import figure

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore') 
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from numpy import random
from random import randrange
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
import time

THRESH_HOLD=.05
EPOCHS = 50 #number of passes of whole data
BATCH_SIZE = 64 #size of data going through at once
LEARNING_RATE = 0.01
NEURONS=5
XSIZE=1000
NUMBER_ITERATIONS=10



    
    
def get_Xvals():
    return np.random.normal(0, 1, size=(10000, 5))

    
    
    
def getdata_XY():
    X=np.random.normal(0, 1, size=(10000, 5))
    test_data = testData(torch.FloatTensor(X))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist =[]
    
    
    GroundTruth.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = GroundTruth(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return X, y_truthlist

def get_yvals(model, X_data):
    model.eval()
    test_data = testData(torch.FloatTensor(X_data))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist=[]
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = model(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return y_truthlist



#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
def binary_acc(y_pred, y_test): 
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

def trainnetwork(network, train_loader,Time=True):
    t0=time.time()
    network.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)


    network.train()
    for e in range(1, EPOCHS+1):
        epoch_loss = 0
        epoch_acc = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            #X_batch = X_batch.astype(np.float32)
            
            
            
            
            y_pred = network(X_batch)
            #y_pred = network(X_batch)

            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f}')

            
    t1=time.time()
    return(t1-t0)
        
        
def check_threshhold(network, test_loader, truth):
    total_loss=0
    network.eval()
    with torch.no_grad():
        for (X_batch, y) in zip(test_loader, truth):
            X_batch = X_batch.to(device)
            y_test_pred = network(X_batch)
            y_pred_tag = torch.sigmoid(y_test_pred)
            #print("y ", y)
            #print("pred ", y_pred_tag.cpu().numpy())
            total_loss=total_loss+((y_pred_tag.cpu().numpy()-y))**2
    
    
    #total_loss=np.sqrt(total_loss)/len(truth)
    total_loss=total_loss/len(truth)
    print("current loss: ", total_loss)
    if total_loss[0][0]<THRESH_HOLD:
        return True
    else:
        return False
        
        
def evalnetwork(network, test_loader, stats=True):
    y_pred_list =[]
    network.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            
            y_test_pred = network(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_pred_tag = torch.round(y_test_pred)
            y_pred_list.append(y_pred_tag.cpu().numpy())


    y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
    if(stats):
        statistics(y_pred_list)
    #return y_fullmodelpred_list
    
    
    


def statistics(y_pred_list):
    correctcounter=0
    for i in range(len(y_pred_list)):
        if y_pred_list[i]==y_test[i]:
            correctcounter=correctcounter+1       

    print("# correct: ",correctcounter, " out of ", len(y_test))

    print("Model statistics")
    print(classification_report(y_test, y_pred_list))    
    
#model class 5 layers, then an output layer
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(5, NEURONS) 
        self.layer_2 = nn.Linear(NEURONS, NEURONS) 
        self.layer_3 = nn.Linear(NEURONS, NEURONS)
        self.layer_4 = nn.Linear(NEURONS,NEURONS)
        self.layer_5 = nn.Linear(NEURONS, NEURONS) 
        self.layer_out = nn.Linear(NEURONS, 1) #output layer
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        
        #self.batchnorm1 = nn.BatchNorm1d(NEURONS)
        #self.batchnorm2 = nn.BatchNorm1d(NEURONS)
        #self.batchnorm3 = nn.BatchNorm1d(NEURONS)
        #self.batchnorm4 = nn.BatchNorm1d(NEURONS)
        #self.batchnorm5 = nn.BatchNorm1d(NEURONS)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        #x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        #x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        #x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        #x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        #x = self.batchnorm5(x)        
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x
     
    def get_copy(self):
        copied_model=binaryClassification()
        copied_model.layer_1 = copy.deepcopy(self.layer_1)
        copied_model.layer_2 = copy.deepcopy(self.layer_2) 
        copied_model.layer_3 = copy.deepcopy(self.layer_3)
        copied_model.layer_4 = copy.deepcopy(self.layer_4)
        copied_model.layer_5 = copy.deepcopy(self.layer_5)
        copied_model.layer_out = copy.deepcopy(self.layer_out)
        return copied_model
    
    def change_node_model(self, node):
        changed_node=binaryClassification()
        changed_node.layer_1 = copy.deepcopy(self.layer_1)
        changed_node.layer_2 = copy.deepcopy(self.layer_2) 
        changed_node.layer_3 = copy.deepcopy(self.layer_3)
        changed_node.layer_4 = copy.deepcopy(self.layer_4)
        changed_node.layer_5 = copy.deepcopy(self.layer_5)
        changed_node.layer_out = copy.deepcopy(self.layer_out) 
        
        weightchange=np.random.normal(0, 1, size=5)
        weightchange=[i.item() for i in weightchange]

        if node[0]==0:
            self.layer_1.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_1.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)

        elif node[0]==1:
            self.layer_2.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_2.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
   
        elif node[0]==2:
            if node[1]==0:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                               
        elif node[0]==3:
            if node[1]==0:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
             
        elif node[0]==4:
            if node[1]==0:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
     
        else:
            newweight=self.layer_output.weight.requires_grad_(False)+torch.tensor([weightchange],requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_output.weight=torch.nn.Parameter(newweight)
                
                
        return changed_node
    
    
    
    
    
    
    
    
    
    def change_node_modellinear(self, node):
        changed_node=binaryClassification()
        changed_node.layer_1 = copy.deepcopy(self.layer_1)
        changed_node.layer_2 = copy.deepcopy(self.layer_2) 
        changed_node.layer_3 = copy.deepcopy(self.layer_3)
        changed_node.layer_4 = copy.deepcopy(self.layer_4)
        changed_node.layer_5 = copy.deepcopy(self.layer_5)
        changed_node.layer_out = copy.deepcopy(self.layer_out) 
        
        weightchange=[1.0]*5

        if node[0]==0:
            self.layer_1.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_1.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_1.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_1.weight=torch.nn.Parameter(newweight)

        elif node[0]==1:
            self.layer_2.weight.requires_grad_(False)
            if node[1]==0:
                newweight=self.layer_2.weight+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_2.weight+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_2.weight=torch.nn.Parameter(newweight)
   
        elif node[0]==2:
            if node[1]==0:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_3.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_3.weight=torch.nn.Parameter(newweight)
                               
        elif node[0]==3:
            if node[1]==0:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_4.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_4.weight=torch.nn.Parameter(newweight)
             
        elif node[0]==4:
            if node[1]==0:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==1:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)  
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==2:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0],[0,0,0,0,0]],requires_grad=False)          
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==3:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange,[0,0,0,0,0]],requires_grad=False)       
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
                
            if node[1]==4:
                newweight=self.layer_5.weight.requires_grad_(False)+torch.tensor([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],weightchange],requires_grad=False)           
                newweight.requires_grad_()
                changed_node.layer_5.weight=torch.nn.Parameter(newweight)
     
        else:
            newweight=self.layer_output.weight.requires_grad_(False)+torch.tensor([weightchange],requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_output.weight=torch.nn.Parameter(newweight)
                
                
        return changed_node
                   
    
    
    
    
                   
def select_node():
    layer=randrange(5)
    if layer!=6:
        node=randrange(5)
    else:
        node=randrange(2)
    return [layer, node]
    
       
#groundtruth model  
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
GroundTruth = binaryClassification()
firstmodel=binaryClassification()




test_model=binaryClassification()
firstmodeltrained=test_model.get_copy()
accepted=False
current_time=0
data_list=[]
while(not accepted):
    X, y=getdata_XY()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
    data_list.append(X)
    current_time=current_time+trainnetwork(firstmodeltrained, train_loader)
    accepted=check_threshhold(firstmodeltrained, test_loader, y_test)
    print(len(data_list))
    
    


    
    
print(current_time)
    
    
    



current loss:  [[0.00116517]]
1
13.120520830154419


In [ ]:
#add Guassian weight
firstpass=True
time_increase=1
while(time_increase>.01):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    for i in range(0, 10):
        train_testmodel=test_model.get_copy()
        prospective_GroundTruth=GroundTruth.change_node_model(node_list[i])
        model_list.append(prospective_GroundTruth)
        data_counter=0
        test_time=0
        inital_data_length=len(data_list)
        accepted=False






        while(not accepted): 
            #use previously generated data
            if data_counter<inital_data_length:
                X=data_list[data_counter]
                y=get_yvals(prospective_GroundTruth, X)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                accepted=check_threshhold(train_testmodel, test_loader, y_test)
                data_counter=data_counter+1
            else:                          
                X=get_Xvals()
                data_list.append(X)
                y=get_yvals(prospective_GroundTruth, X)



                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1)
                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                accepted=check_threshhold(train_testmodel, test_loader, y_test)

            #print(test_time)
        time_list.append(test_time)
    
    
    print(time_list)
    max_new_time=max(time_list)
    max_index=time_list.index(max_new_time)
    GroundTruth=model_list[i]
    time_increase=max_new_time-current_time
    current_time=max_new_time

    
    

                      
                          
        
                                 
        
    
    
    
    
    





In [2]:
#add [1]*5
firstpass=True
time_increase=1
while(time_increase>.01):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    for i in range(0, 10):
        train_testmodel=test_model.get_copy()
        prospective_GroundTruth=GroundTruth.change_node_modellinear(node_list[i])
        model_list.append(prospective_GroundTruth)
        data_counter=0
        test_time=0
        inital_data_length=len(data_list)
        accepted=False






        while(not accepted): 
            #use previously generated data
            if data_counter<inital_data_length:
                X=data_list[data_counter]
                y=get_yvals(prospective_GroundTruth, X)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                accepted=check_threshhold(train_testmodel, test_loader, y_test)
                data_counter=data_counter+1
            else:                          
                X=get_Xvals()
                data_list.append(X)
                y=get_yvals(prospective_GroundTruth, X)



                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                test_data = testData(torch.FloatTensor(X_test))
                train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                test_loader = DataLoader(dataset=test_data, batch_size=1)
                test_time=test_time+trainnetwork(train_testmodel, train_loader)
                accepted=check_threshhold(train_testmodel, test_loader, y_test)

            #print(test_time)
        time_list.append(test_time)
    
    
    print(time_list)
    max_new_time=max(time_list)
    max_index=time_list.index(max_new_time)
    GroundTruth=model_list[i]
    time_increase=max_new_time-current_time
    current_time=max_new_time

    
    


current loss:  [[0.00016184]]
current loss:  [[0.00111658]]
current loss:  [[0.00096108]]
current loss:  [[0.00114861]]
current loss:  [[0.00110419]]
current loss:  [[0.00043214]]
current loss:  [[0.00111681]]
current loss:  [[0.0010474]]
current loss:  [[0.00113417]]
current loss:  [[0.00110339]]
[12.976532697677612, 12.938209056854248, 14.750714302062988, 12.007935762405396, 11.025629043579102, 10.595731019973755, 12.371632814407349, 11.334189653396606, 13.096515417098999, 12.43299150466919]
current loss:  [[5.5408644e-05]]
current loss:  [[0.00071849]]
current loss:  [[0.00042576]]
current loss:  [[0.00038151]]
current loss:  [[0.00106397]]
current loss:  [[0.00112753]]
current loss:  [[0.0007163]]
current loss:  [[0.00146907]]
current loss:  [[0.00049282]]
current loss:  [[0.00100258]]
[13.589966773986816, 12.514455795288086, 12.277449607849121, 12.588509798049927, 13.095532655715942, 12.446441650390625, 12.127474069595337, 13.397139310836792, 12.944106101989746, 12.364925622940063